#### PROBLEM 7: DBSCAN on real data
Run the DBSCAN algorithm on the 20NG dataset, and on the FASHION dataset, and the HouseHold dataset (see papers), and evaluate results. You need to implement both phases (1) neighborhoods creation, (2) DBSCAN.
Explain why/when it works, and speculate why/when not. You need to trial and error for parameters epsilon and MinPts

DBSCAN Revisited, Revisited: Why and How You Should (Still) Use DBSCAN
DBSCAN Revisited:Mis-Claim, Un-Fixability, and Approximation

EXTRA CREDIT: Using class labels (cheating), try to remove/add points in curate the set for better DBSCAN runs

In [1]:
import pandas as pd
import numpy as np
import random
from collections import Counter
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score

In [57]:
def get_nearest_neighbor(core_vector, X, eps, core_vector_index):
    nearest_neighbor = []
    euclidian_distances = euclidean_distances(X)

    for i in range(len(euclidian_distances)):
        if i != core_vector_index:
            euclidean_distance = euclidian_distances[core_vector_index][i]
            if(euclidean_distance <= eps):
                nearest_neighbor.append(i)
    
    return nearest_neighbor


def check_core_point(eps,minPts, X, index):

    nearest_neighbor = list(get_nearest_neighbor(X[index], X, eps, index))
    
    if len(nearest_neighbor) >= minPts:
        return (nearest_neighbor, 1)
    
    elif (len(nearest_neighbor) < minPts) and len(nearest_neighbor) > 0:
        return (nearest_neighbor, 2)
    
    elif len(nearest_neighbor) == 0:
        return (nearest_neighbor, 3)

In [3]:
def dbscan(eps, minPts, X):
    
    #initiating cluster number
    cluster_num = 0

    q = set()
    unvisited = [i for i in range(1, len(X))]
    clusters = []
    
    
    while (len(unvisited) > 0): #run until all points have been visited

        #identifier for first point of a cluster
        first_point = True
        
        #choose a random unvisited point
        q.add(random.choice(unvisited))
        
        while len(q) > 0:
            pop = q.pop()
            unvisited.remove(pop)
            
            neighbor_ind, point_type = check_core_point(eps, minPts, X, pop)
            
            #dealing with an edge case
            if point_type == 2 and first_point:
                
                clusters.append((pop, -1))
                for ind in neighbor_ind:
                    clusters.append((ind, -1))

                unvisited = [element for element in unvisited if element not in neighbor_ind]
                continue

            first_point = False
            
            #CORE POINT
            if point_type == 1:
                clusters.append((pop,cluster_num))
                neighbor_ind = set(neighbor_ind) & set(unvisited)
                q.update(neighbor_ind)

            #BORDER POINT
            elif point_type == 2:
                clusters.append((pop,cluster_num))
                continue
            
            #OUTLIER
            elif point_type == 3:
                clusters.append((pop, -1))
                continue
                
        if not first_point:
            cluster_num += 1
        
    return clusters

In [17]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

vectorizer = TfidfVectorizer(stop_words='english')

train_data_vector = vectorizer.fit_transform(newsgroups_train.data).toarray()
test_data_vector = vectorizer.fit_transform(newsgroups_test.data).toarray()

train_labels = newsgroups_train.target
test_labels = newsgroups_test.target

random_sample_indices = random.sample(range(train_data_vector.shape[0]), 3000)
train_data_20 = train_data_vector[random_sample_indices]
train_labels_20 = train_labels[random_sample_indices]

train_data_final_80, validation_data_final_10, train_labels_final_80,validation_labels_final_10 = train_test_split(train_data_20, train_labels_20, test_size=0.1, random_state=42)

In [28]:
count = 0
for i in range(len(validation_data_final_10)):
    for j in range(len(validation_data_final_10[i])):
        if validation_data_final_10[i][j] > 0:
            count +=1
print(count)

31899


In [60]:

clustered = dbscan(13, 4, validation_data_final_10)
idx , cluster = list(zip(*clustered))
cluster_df = pd.DataFrame(clustered, columns = ["ind", "cluster"])
print(np.unique(cluster))

print(len(clustered))
print((clustered))

[0]
299
[(38, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0), (46, 0), (47, 0), (48, 0), (49, 0), (50, 0), (51, 0), (52, 0), (53, 0), (54, 0), (55, 0), (56, 0), (57, 0), (58, 0), (59, 0), (60, 0), (61, 0), (62, 0), (63, 0), (64, 0), (65, 0), (66, 0), (67, 0), (68, 0), (69, 0), (70, 0), (71, 0), (72, 0), (73, 0), (74, 0), (75, 0), (76, 0), (77, 0), (78, 0), (79, 0), (80, 0), (81, 0), (82, 0), (83, 0), (84, 0), (85, 0), (86, 0), (87, 0), (88, 0), (89, 0), (90, 0), (91, 0), (92, 0), (93, 0), (94, 0), (95, 0), (96, 0), (97, 0), (98, 0), (99, 0), (100, 0), (101, 0), (102, 0), (103, 0), (104, 0), (105, 0), (106, 0), (107, 0), (108, 0), (109, 0), (110, 0), (111, 0), (112, 0), (113, 0), (114, 0), (115, 0

purity index
gini index
siluvette score

In [43]:
def calculate_gini(labels):
    _, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    gini_index = 1 - np.sum(probabilities ** 2)
    return gini_index

In [44]:
calculate_gini(cluster)

0.0

In [49]:
cluster = list(cluster)
while len(cluster) > len(validation_data_final_10):
    cluster.pop()

In [62]:
cluster_member_map = dict()
member_cluster_map = dict()

for pair in clustered:
    member = pair[0]
    cluster_num = pair[1]

    if cluster_num in cluster_member_map:
        cluster_member_map[cluster_num].add(member)
        member_cluster_map[member] = cluster_num
    else:
        cluster_member_map[cluster_num] = set()
        cluster_member_map[cluster_num].add(member)

cluster_label_map = dict()

for key in cluster_member_map:
    members = cluster_member_map[key]
    actual_labels = []
    for member in members:
        label = validation_labels_final_10[member]
        # print("label: ", label)
        actual_labels.append(label)
    mode = Counter(actual_labels).most_common(1)
    cluster_label_map[key] = mode[0][0]



pred_labels = []

for key in member_cluster_map:
    label = cluster_label_map[member_cluster_map[key]]
    pred_labels.append(label)

  
# print(cluster_label_map)
# print((true_labels))

In [63]:
while len(pred_labels) < len(validation_data_final_10):
    pred_labels.append(0)

In [64]:
purity = accuracy_score(pred_labels, validation_labels_final_10)
print("Purity: ", purity)

Purity:  0.08
